<a href="https://colab.research.google.com/github/inyminymoe/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%EA%B8%B0%EB%B0%98%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D(%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC)_%EA%B0%95%EC%9D%B8%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API 호출

In [4]:
# 설치 및 import
!pip install pandas requests

import requests
import pandas as pd

In [6]:
# API 설정
api_key = '7657646572696e7931313561474d6b41'
year, month = 2024, 12
url = 'http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/'

# API 호출
response = requests.get(url)

# 데이터 출력
print('URL:', url )
print('status_code:', response.status_code)
print('Content-Type:', response.headers.get('Content-Type'))

# 데이터(JSON) 확인
data = response.json()
print('data:', data)
rows = data['energyUseDataStatInfo']['row']

# 호출 성공/실패 출력
try:
    response.raise_for_status()
    print('API 호출 성공')

except requests.exceptions.HTTPError as e:
    print('API 호출 실패:', e)

df = pd.DataFrame(rows)
print(df.head())

URL: http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/
status_code: 200
Content-Type: application/json;charset=UTF-8
data: {'energyUseDataStatInfo': {'list_total_count': 4110, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2024', 'MON': '12', 'HOSTCODE': '강남구', 'DONGCODE': '기타', 'MM_TYPE': '학교', 'TYPE_DIV': '단체', 'TYPE_DIV_CD': 'TB_GROUP', 'CNT': '1', 'EUS': '40385', 'EUS1': '38786', 'EUS2': '38383', 'ECO2_1_MATH_YEAR': '1599', 'ECO2_2_MATH_YEAR': '677.976', 'ECO2_1_MATH': '1800.5', 'ECO2_2_MATH': '763.412', 'ECO2_1': '1800.5', 'ECO2_2': '1800.5', 'E_TOE_YEAR': '0.36777', 'E_RATE_YEAR': '4.123', 'E_TOE_AVG': '0.414', 'E_RATE_AVG': '4.666', 'GUS': '3471', 'GUS1': '6841', 'GUS2': '5855', 'GCO2_1_MATH_YEAR': '-3370', 'GCO2_2_MATH_YEAR': '-7548.8', 'GCO2_1_MATH': '-2877', 'GCO2_2_MATH': '-6444.48', 'GCO2_1': '-2877', 'GCO2_2': '-5754', 'G_TOE_YEAR': '-3.515', 'G_RATE_YEAR': '-49.262', 'G_TOE_AVG': '-3.001', 

In [12]:
# API 및 기간 설정
api_key = '7657646572696e7931313561474d6b41'
endpoint = 'energyUseDataSummaryInfo'
YEARS = range(2015, 2025)
start, end = 1, 1000
data_list = []

def build_url(y:int, m:int,) -> str:
    return f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{endpoint}/{start}/{end}/{year}/{month:02d}"

def is_personal(row):
    if str(row.get("TYPE_DIV","")).strip() == "개인":
        return True
    cd = str(row.get("TYPE_DIV_CD", "")).upper()
    return("PERSON" in cd) or ("TB_PERSON" in cd)

# 개인 + 현년만 모으기
all_rows = []
for y in YEARS:
    for m in range(1, 12+1):
        r = requests.get(url(y, m), timeout=30)
        if r.status_code != 200:
            print(f'[{y}-{m:02d}] HTTP {r.status_code}')
            continue
        try:
            rows = r.json().get(endpoint, {}).get('row', []) or []
        except Exception:
            print(f'[{y}-{m:02d}] JSON 파싱 실패')
            continue
        personal = [row for row in rows if is_personal(row)]

        for row in personal:
            row['year'] = y
            row['month'] = m
        print(f'[{y}-{m:02d}] 개인 {len(personal)}건')
        all_rows.extend(personal)

df = pd.DataFrame(all_rows)
print("\n=== (2-1) DataFrame info ===")
df.info()
print("\n=== (2-1) Head ===")
display(df.head())


TypeError: 'str' object is not callable

In [15]:
import requests
import pandas as pd

API_KEY = '7657646572696e7931313561474d6b41'
ENDPOINT = 'energyUseDataSummaryInfo'
YEARS = range(2015, 2025)
START, END = 1, 1000


def url_of(y, m):
    return f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{ENDPOINT}/{START}/{END}/{y}/{m:02d}"

all_rows = []
for y in YEARS:
    for m in range(1, 13):
        try:
            r = requests.get(url_of(y, m), timeout=30)
            if r.status_code != 200:
                print(f"[{y}-{m:02d}] HTTP {r.status_code}")
                continue
            js = r.json()
            rows = js.get(ENDPOINT, {}).get("row", []) or []
            # 메타 추가
            for row in rows:
                row["year"] = y
                row["month"] = m
            all_rows.extend(rows)
        except Exception as e:
            print(f"[{y}-{m:02d}] 스킵: {type(e).__name__}")

df_raw = pd.DataFrame(all_rows)
print("전체 행수:", len(df_raw))
display(df_raw.head())

print("=== 실제 컬럼 목록 ===")
print(sorted(df_raw.columns.tolist()))

# 주요 후보 컬럼들의 유니크 값 분포 살펴보기
for col in ["TYPE_DIV", "TYPE_DIV_CD", "MM_TYPE", "HOSTCODE", "DONGCODE"]:
    if col in df_raw.columns:
        print(f"\n=== {col} 유니크 값(상위 30개) ===")
        vc = df_raw[col].value_counts(dropna=False)
        display(vc.head(30))
    else:
        print(f"\n=== {col} 컬럼 없음 ===")

# '개인'이라는 문자열이 아예 안 나오는지 전역 검색(안전빵)
contains_personal = df_raw.astype(str).apply(lambda s: s.str.contains("개인", na=False))
print("\n'개인' 문자열을 포함하는 셀 개수:", contains_personal.values.sum())

# 샘플 몇 행만 출력해서 감으로 보기
print("\n=== 샘플 3행 ===")
display(df_raw.sample(min(3, len(df_raw))) if len(df_raw) else df_raw)

전체 행수: 840


,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE,year,month
0,2015,01,개인,767791,193784708,194781915,204969429,-6090964,-2582568.736,59133720,...,8179.308,2715.530256,22740838.937,23400055.303,27090493.875,-2504435.652,-33660084.213069,2015-06-04 17:03:55.0,2015,1
1,2015,01,학교,1382,134955565,128707423,145561511,-2178902,-923854.448,9107197,...,-39498.1,-13113.3692,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1
2,2015,01,종교단체,32,372270,363376,411363,-15099.5,-6402.188,53331,...,1411.5,468.618,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1
3,2015,01,소상공인,2058,10249618,9927610,11039593,-233983.5,-99209.004,536119,...,369.9,122.8068,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1
4,2015,01,공동주택관리소,1747,327102096,329852334,351694752,-13671447,-5796693.528,22834274,...,-26645.4,-8846.2728,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1


=== 실제 컬럼 목록 ===
['CNT', 'ECO2_1', 'ECO2_2', 'EUS', 'EUS1', 'EUS2', 'GCO2_1', 'GCO2_2', 'GUS', 'GUS1', 'GUS2', 'HCO2_1', 'HCO2_2', 'HUS', 'HUS1', 'HUS2', 'MM_TYPE', 'MON', 'REG_DATE', 'WCO2_1', 'WCO2_2', 'WUS', 'WUS1', 'WUS2', 'YEAR', 'month', 'year']

=== TYPE_DIV 컬럼 없음 ===

=== TYPE_DIV_CD 컬럼 없음 ===

=== MM_TYPE 유니크 값(상위 30개) ===


,count
MM_TYPE,
개인,120
학교,120
종교단체,120
소상공인,120
공동주택관리소,120
기업,120
공공기관,120



=== HOSTCODE 컬럼 없음 ===

=== DONGCODE 컬럼 없음 ===

'개인' 문자열을 포함하는 셀 개수: 120

=== 샘플 3행 ===


,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE,year,month
11,2015,02,종교단체,32,336476,341646,366928,-17811,-7551.864,46069,...,-291,-96.612,0,0,0,0,0,2015-06-04 16:59:30.0,2015,2
14,2015,03,학교,3903,145577747,140333207,148915260,673170,285424.08,7601585,...,-137612.4,-45687.3168,48,34,41,10.5,41.811,2015-06-12 04:00:01.0,2015,3
375,2019,06,공동주택관리소,3223,408970817,420042269,411858024,-6979329.5,-2959235.708,3475177,...,-118218.65,-39248.5918,0,0,0,0,0,2019-09-01 04:09:05.0,2019,6
